In [1]:
simulation_results_array = Array{Float64,2}(undef, number_of_samples, number_of_samples);
for i ∈ eachindex(β)
    β_outer = β[i]
    
    model = deepcopy(cashflow_check_model);
    model.coupon = β_outer*cashflow_check_model.coupon;
    
    for j ∈ eachindex(β)
        
        β_inner = β[j];
        model.rate = β_inner*cashflow_check_model.rate;
        perturbed_price = model |> discount_model |> x-> x.price
        simulation_results_array[i,j] = ((perturbed_price - nominal_price)/nominal_price)*100
    end
end
simulation_results_array;

LoadError: UndefVarError: `number_of_samples` not defined

In [2]:
# build a pretty table to display the results -
(R,C) = size(simulation_results_array)
pretty_table_data = Array{Any,2}(undef, R, C+1)

# first col holds labels -
for i ∈ 1:R
    if (i == 1)
        pretty_table_data[i,1] = "-20% coupon";
    elseif (i == 3)
        pretty_table_data[i,1] = "+20% coupon";
    else
        pretty_table_data[i,1] = "nominal coupon";
    end
end

for i = 1:R
    for j = 1:C
        pretty_table_data[i,j+1] = simulation_results_array[i,j]
    end
end

header_data = (["", "-20% yield", "nominal yield", "+20% yield"])
pretty_table(pretty_table_data, header=header_data)

LoadError: UndefVarError: `simulation_results_array` not defined